In [378]:
import os
import pandas as pd
import numpy as np

In [379]:
os.listdir('data')

['out.csv', 'samplems.csv', 'testms.csv', 'trainms.csv']

In [380]:
train_data = pd.read_csv('data/trainms.csv')

In [381]:
print(train_data["treatment"])
y = pd.factorize(train_data["treatment"])[0] + 1
del train_data["treatment"]
print(y)

0      Yes
1       No
2       No
3      Yes
4       No
      ... 
995     No
996    Yes
997     No
998     No
999     No
Name: treatment, Length: 1000, dtype: object
[1 2 2 1 2 2 1 2 1 2 1 2 1 2 2 1 1 1 2 2 1 2 1 1 1 1 2 1 1 2 2 2 1 1 1 1 2
 2 2 1 1 2 1 2 1 1 2 2 2 1 1 2 2 2 2 1 2 1 2 1 1 1 2 2 2 2 1 2 2 2 2 2 2 2
 2 2 2 2 1 2 2 2 2 1 2 2 1 1 1 2 2 1 2 2 1 2 2 1 1 1 1 2 2 1 2 2 1 1 2 1 1
 1 1 2 2 2 2 2 1 2 2 2 1 2 1 2 2 1 2 1 1 2 2 2 1 2 2 1 2 2 1 1 2 2 2 1 2 2
 1 2 2 2 2 2 1 1 1 2 1 1 1 1 2 2 1 2 2 2 2 2 1 1 1 2 1 2 2 1 1 2 2 2 2 1 1
 2 2 2 2 1 1 1 2 1 2 2 2 2 2 1 1 1 1 2 2 1 1 1 2 2 2 2 1 1 1 1 1 2 1 2 2 2
 2 1 2 2 2 2 2 1 1 1 1 2 2 2 1 2 1 1 2 1 1 2 2 1 1 2 1 1 1 2 1 2 2 2 1 2 2
 1 1 2 1 1 2 2 1 1 1 2 2 1 2 2 1 2 2 2 1 2 1 2 1 1 2 1 1 2 1 1 2 2 1 2 1 2
 1 2 1 2 1 1 2 1 1 1 1 2 2 2 1 1 1 1 2 2 2 2 1 2 1 1 2 1 1 2 2 1 2 1 2 1 2
 1 2 2 1 2 2 2 2 2 2 2 2 1 1 1 2 1 2 1 2 1 2 1 1 1 1 2 2 1 1 1 1 1 2 1 1 1
 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 2 2 2 2 2 1 1 1 2 1 1 2 1
 2 2 2 1 

In [382]:
def drop_irrelvant_cols(data):
    del data["s.no"]
    del data["comments"]
    del data["Timestamp"]
    del data["state"]
    return data

In [383]:
train_data = drop_irrelvant_cols(train_data)

In [384]:
train_data.head()

,Age,Gender,Country,self_employed,family_history,work_interfere,no_employees,remote_work,tech_company,benefits,...,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,37,Female,United States,NaN,No,Often,6-25,No,Yes,Yes,...,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No
1,44,M,United States,NaN,No,Rarely,More than 1000,No,No,Don't know,...,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No
2,32,Male,Canada,NaN,No,Rarely,6-25,No,Yes,No,...,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No
3,31,Male,United Kingdom,NaN,Yes,Often,26-100,No,Yes,No,...,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes
4,31,Male,United States,NaN,No,Never,100-500,Yes,Yes,Yes,...,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No


In [385]:
def remove_age_outliers(data):
    avg = np.average(data)
    for index, age in enumerate(data):
        if age < 18 or age > 70:
            data[index] = avg
    return data
train_data["Age"] = remove_age_outliers(train_data["Age"])

In [386]:
from sklearn.neural_network import MLPClassifier

In [387]:
clf = MLPClassifier(solver='adam', hidden_layer_sizes=(27, 54, 2), random_state=0, early_stopping=True, validation_fraction=0.2, learning_rate='adaptive')

In [388]:
def categorize_cols(data):
    categorical_columns = ["anonymity", "benefits", "care_options", "Country", "coworkers", 
                           "Gender", "leave", "mental_health_consequence", "mental_health_interview", 
                           "mental_vs_physical", "no_employees", "phys_health_consequence", "phys_health_interview", 
                           "seek_help", "supervisor", "wellness_program", "work_interfere", "obs_consequence", 
                           "tech_company", "remote_work", "family_history", "self_employed"]

    for col in categorical_columns:
        data[col] = pd.factorize(data[col])[0] + 1
    return data

In [389]:
train_data = categorize_cols(train_data)

In [390]:
clf.fit(train_data, y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(27, 54, 2), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=0, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.2, verbose=False, warm_start=False)

In [391]:
test_data = pd.read_csv('data/testms.csv')

In [392]:
test_data = drop_irrelvant_cols(test_data)

In [393]:
test_data = categorize_cols(test_data)

In [394]:
pred = clf.predict(test_data)

In [395]:
result = [[index+1, "Yes"] if d == 0 else [index+1, "No"] for index, d in enumerate(pred)]

In [396]:
import csv

with open("data/out.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["s.no", "treatment"])
    writer.writerows(result)

In [397]:
import pandas_profiling
test_data = pd.read_csv('data/testms.csv')

In [398]:
test_data.profile_report()

In [399]:
import pickle

In [400]:
os.listdir()

['.ipynb_checkpoints',
 'data',
 'dataCleanUp.ipynb',
 'model.pkl',
 'Untitled.ipynb']

In [401]:
f = open('model.pkl', 'rb')

In [403]:
p = pickle.load(f)

In [406]:
p.predict(y)

AttributeError: 'int' object has no attribute 'predict'